In [6]:
# La versione deve essere < 2.x SE NO NON FUNZIONA!!
%tensorflow_version 1.x
# Load the TensorBoard notebook extension.
%load_ext tensorboard
import tensorflow as tf
tf.test.gpu_device_name()
print(tf.__version__)
!pip install --no-deps keras==2.2.4

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
1.15.2
  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [7]:
import os
# Clone Mask_RCNN github repository with data
if not os.path.isdir('./Mask_RCNN'):
  !git clone https://github.com/simo-cava/Mask_RCNN

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 960 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (960/960), 76.49 MiB | 31.66 MiB/s, done.
Resolving deltas: 100% (616/616), done.


In [8]:
# install mrcnn library
%cd Mask_RCNN
!pip install -r requirements.txt
# !pip install --no-deps keras==2.2.4
!python setup.py install
#%cd ..
!pip show mask-rcnn

# DOPO aver installato le librerie riavvio il runtime perchè alcune ne hanno bisogno 

/content/Mask_RCNN/Mask_RCNN
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> build

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from numpy import expand_dims
from numpy import mean
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "test_face_mask_cfg"
	# Train on 1 GPU and 32 images per GPU. We can put multiple images on each
  # GPU because the images are small. Batch size is 32 (GPUs * images/GPU).
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	# number of classes (background + #classes)
	NUM_CLASSES = 1 + 3

  # Use smaller anchors because our image and objects are small
	
	IMAGE_MAX_DIM = 1024
 

In [11]:
# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model.load_weights('/content/drive/MyDrive/MaskRCNN weights/MaskDetectionFinal/lr=0.002_Equal_Loss_20epoch(head)+10epoch(5+)/face_mask_cfg20210210T0817/mask_rcnn_face_mask_cfg_0030.h5', by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.






Re-starting from epoch 30


## Run Face Mask Detection on Images


In [49]:
import numpy as np
from shapely.geometry import Polygon
#take the image and apply the box, and Label
def display_instances_suppr(image, boxes, masks, ids, names, scores, overlap = 80):
 
  N = boxes.shape[0]
  # Generate BGR colors  green, red, yellow
  colors = [[0,255,0],[0,0,255],[0, 255, 255]]

  if not N:
    print('NO INSTANCES TO DISPLAY')
  else:
    assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

  invalid_bb = []
  for i in range(N-1):
    # Bounding box
    if not np.any(boxes[i]):
      # Skip this instance. Has no bbox. Likely lost in image cropping.
      continue
    if i in invalid_bb:
      continue

    y1a, x1a, y2a, x2a = boxes[i]
    for j in range(i+1,N):
      # suppresion of overlapped boundinbox with different labels on the same face
      intersect = 0
      swap = False
      
      # creation of Plygons for intersec calculation
      box_1 = [(x1a,y1a), (x2a,y1a), (x2a,y2a), (x1a,y2a)]
      y1b, x1b, y2b, x2b = boxes[j]
      box_2 = [(x1b,y1b), (x2b,y1b), (x2b,y2b), (x1b,y2b)]
      poly_1 = Polygon(box_1)
      poly_2 = Polygon(box_2)
      
      # max box = box_1 --> area min = poly_2
      if poly_2.area < poly_1.area:
        poly_min_area = poly_2.area
      else:
        # max box = box_2 --> area min = poly_1
        poly_min_area = poly_1.area
      
      # calculate intersection of bb
      intersect = (poly_1.intersection(poly_2).area/poly_min_area)*100
      #print(intersect)

      # found intersection so skip the i-th boundingbox
      if intersect > overlap:
        # need swap for succesive comparison, so we don't show the i-th boundingbox
        if scores[i] > scores[j]:
          invalid_bb.append(j)
        else:
          invalid_bb.append(i)
          break

  for i in range(N):
    if not np.any(boxes[i]):
      continue
    if i in invalid_bb:
        continue
    color = colors[ids[i]-1]
    y1, x1, y2, x2 = boxes[i]
    label = names[ids[i]]
    score = scores[i] if scores is not None else None
    caption = '{} {:.2f}'.format(label, score) if score else label
    image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  return image

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow
# path 
path = '/content/drive/MyDrive/Images/img5.jpg'
name = 'img5'
name = name+'_pred.jpg'
# Using cv2.imread() method 
img = cv2.imread(path)
cv2_imshow(img) 
img = img[None, :, : ,:]
r = model.detect(img, verbose=0)[0]
img = img[-1,:,:,:]
img_predicted = display_instances_suppr(img, r['rois'], r['masks'], r['class_ids'], ['BG', 'with_mask', 'without_mask', 'mask_weared_incorrect'], r['scores'], overlap = 50)
# Displaying the image 
name = os.path.join('/content/drive/MyDrive/Images', name)
cv2.imwrite(name, img_predicted)
cv2_imshow(img_predicted) 